分类模型的损失函数有很多种,比如
- svm的hinge loss
- 逻辑回归的logit loss
- MSE
- 多分类时的交叉熵

# 多分类模型的设定

多分类模型下,一共有K个类,真实概率分布为$(y^1, y^2, ..., y^K)$,预测概率分布为$(\hat{y}^1, \hat{y}^2, ..., \hat{y}^K) $.样本量为N.

# 均方损失/MSE (Mean Square Error)
基于MSE的损失函数为  
$$L = \frac{1}{N} \sum_{i=1}^{N} ||y_i - \hat{y}_i ||_2^2  $$
其中$y_i, \hat{y}_i$分别是第i个样例的真实值和预测值,可以是向量. $|| a - b ||_2^2$表示a和b的[L2距离或欧几里得距离](https://en.wikipedia.org/wiki/Euclidean_distance).

多分类模型下损失函数则为 
<span id="equation1">
$$L = \frac{1}{N} \sum_{i=1}^{N} \sum_{k=1}^K  (y_i^k - \hat{y}_i^k)^2  \tag{1}$$ 
</span>


# 交叉熵方法 (Cross-entropy method)
交叉熵方法基于信息论中[交叉熵](https://en.wikipedia.org/w/index.php?title=Cross_entropy&oldid=491011306)的概念,反映预测概率分布和真实概率分布的差别.  

多分类模型下,基于交叉熵的损失函数为
$$ L = \frac{1}{N} \sum_{i=1}^{N} \sum_{k=1}^{K} y_i^k * log \hat{y}_i^k $$

往往每个样例的真实概率分布为one-hot型,也就是说每个样例都属于某类，$y_i^j=  \begin{cases} 0, j \neq k_i \\  1, j = k_i \end{cases}  \forall i \in \{1, ..., N\}$.所以上式等于  <span id="equation2">
$$ L = \frac{1}{N} \sum_{i=1}^{N} log \hat{y}_i^{k_i} \tag{2}$$  </span>
其中$k_i$为第i个样例的类别.  


在二分类模型中,不妨说类别是1和0, 也就是$y_i \in \{1, 0\}, \forall i \in \{1, 2, ..., N\}$.一般来说,模型的预测值$\hat{y}_i$是样例为1类的预测概率.这样我们有
$$ log \hat{y}_i^{k_i} = \begin{cases} log \hat{y}_i, y_i=1 \\  log (1-\hat{y}_i),  y_i=0  \end{cases}$$

不论$k_i$是0还是1, $$log \hat{y}_i^{k_i} = y_i*log\hat{y}_i + (1-y_i) * log (1-\hat{y_i})$$
[公式(2)](#equation2)等于
$$  \frac{1}{N} \sum_{i=1}^{N} y_i*log\hat{y}_i + (1-y_i) * log (1-\hat{y_i})$$
是不是很熟悉?
比如逻辑回归用的是log odds模型,也就是 
$$ \hat{y}_i = \frac{1}{1+ exp^{-(wx_i+b)}}$$
基于cross-entropy的损失函数就变成了[logit loss](../BasicNotions.ipynb).

# 梯度

按照链式法则，我们有
$$\frac{\partial{L}}{\partial{\theta}} = \frac{1}{N} \sum_{i=1}^N \frac{\partial{L}} {\partial{\hat{y_i}}}   \frac{\partial{\hat{y}_i}} {\partial{\theta}}$$
其中$\theta$是模型的参数,比如逻辑回归中的权重w和偏置b

梯度的**项一**$\frac{\partial{\hat{y}_i}} {\partial{\theta}}$和模型有关,也就是和$\hat{y}_i$的计算方式有关.  
**项二**$\frac{\partial{L}} {\partial{\hat{y}}}$和损失函数用的是MSE还是交叉熵有关.
- 用MSE时,
<span id="item2_MSE">
$$ \frac{\partial{L}} {\partial{\hat{y}_i}} = 2 (\hat{y}_i - y_i)  $$ 
</span>

- 用交叉熵时，
<span id="item2_CE">
$$ \frac{\partial{L}} {\partial{\hat{y}_i}} = \frac{y_i}{\hat{y}_i} $$
</span>


# 分类模型的梯度  

Recall that在分类模型中真实概率分布为$(y^1, y^2, ..., y^K)$,预测概率分布为$(\hat{y}^1, \hat{y}^2, ..., \hat{y}^K) $.当残差$\hat{y} - y$的绝对值很小时,

- 用MSE时的[项二](#item2_MSE)和残差成正比,不仅恒小于1还会很小
- 用交叉熵时,[项二](#item2_CE)中对应k类的分量$ = \begin{cases} \frac{1}{\hat{y}_i^k} (\text{a little bit greater than 1}),  y_k =1 \\ 0, y_k =0 \end{cases}$

优化的后期往往会残差很小,如果使用的优化算法学习率不能自适应或者优化算法只用一阶梯度信息,则MSE损失函数的收敛可能会慢很多.

# 分类模型中交叉熵比MSE更合适
- 从[公式(1)](#equation1)可以看出,MSE无差别得关注全部类别上预测概率和真实概率的差. 
- 从[公式(2)](#equation2)得知,交叉熵关注的是正确类别的预测概率.

分类问题中,模型的输出空间是概率分布,但目标输出空间是样例的类别,也就是说我们最终目标是获得正确的类别.

因为在损失函数计算时每个样例是独立的,我们不妨从单个样例的角度来看MSE和交叉熵的区别.

1. 如果真实标签是(1, 0, 0),模型1的预测标签是(0.8, 0.2, 0),模型2的是(0.8, 0.1, 0.1),那么MSE-based,就是模型2更好;交叉熵-based认为一样.从最终预测的类别上看,模型1和模型2的真实输出其实是一样的.

2. 再换个角度,MSE对残差大的样例惩罚更大些.比如真实标签分别是(1, 0, 0).模型1的预测标签是(0.8, 0.2, 0),模型2的是(0.9, 0.1, 0).即使输出的标签都是类别0, 但MSE-based算出来模型1的误差是模型2的4倍,而交叉熵-based算出来模型1的误差是模型2的2倍左右.为了弥补模型1在这个样例上的损失,MSE-based需要3个完美预测的样例才能达到和模型2一样的损失,而交叉熵-based只需要一个.实际上,模型输出正确的类别,0.8可能已经是个不错的概率了.

PS: MSE是假设数据符合高斯分布时的交叉熵,详情参看樱花书深度学习的5.5.1节.